# Summarization

In [2]:
import torch
from transformers import pipeline
from transformers import AutoModel, AutoTokenizer, BertTokenizer, BertModel, AutoModelForSeq2SeqLM, BartForConditionalGeneration, AutoModelForSequenceClassification, AutoModelForTokenClassification
from collections import Counter
import googletrans

B:\Software\Anaconda\envs\predixion\lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
translate = googletrans.Translator()

In [4]:
en_conversations = []
hi_conversations= []
en_agent_spoke = []
hi_agent_spoke = []

with open('convo.txt', 'r',encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if line:
            speaker, dialogue = line.split(':')
            en_dialogue = translate.translate(dialogue.strip(),dest="en").text
            hi_conversations.append(dialogue)
            en_conversations.append(f'{en_dialogue}')
            if(speaker == 'Speaker 1'):
                en_agent_spoke.append(en_dialogue)
                hi_agent_spoke.append(dialogue)

hi_convo = '| '.join(hi_conversations)
en_convo = '. '.join(en_conversations)

In [5]:
#To split conversations into smaller chunks (for model token limit)

en_chunks = []
context_window = 10
for i in range(0,len(en_conversations),context_window):
    sentences = en_conversations[i:min(i + context_window, len(en_conversations))]
    _ = ' '.join(sentences)
    en_chunks.append(_)

hi_chunks = []
context_window = 10
for i in range(0,len(hi_conversations),context_window):
    sentences = hi_conversations[i:min(i + context_window, len(hi_conversations))]
    _ = ' '.join(sentences)
    hi_chunks.append(_)

In [6]:
#Minimum no. of Tokens in summary 
min_response = 100

#Maximum no. of Tokens in summary
max_response = 300

In [12]:
summary_tokenize = AutoTokenizer.from_pretrained("kabita-choudhary/finetuned-bart-for-conversation-summary")
summarize = AutoModelForSeq2SeqLM.from_pretrained("kabita-choudhary/finetuned-bart-for-conversation-summary")

In [14]:
inputs = summary_tokenize([en_convo], max_length=1024, return_tensors="pt")
summary_ids = summarize.generate(inputs["input_ids"], num_beams=2, min_length=min_response, max_length=max_response)
summary = summary_tokenize.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [16]:
print(f'Conversation Summary in English: \n{summary}\n')
hi_summary = translate.translate(summary,dest='hi').text
print(f'Conversation Summary in Hindi: \n{hi_summary}')

Conversation Summary in English: 
Suresh owes CFI Finance Company Rs 75,000 in outstanding EMI. He owes Rs 50,000 to the company. CFI wants Rs 25,000 by tomorrow. Suresh's mother is ill and the money was spent on her treatment. If he doesn't pay the money by tomorrow, CFI will come to his house and he will call the police and they will see who can treat him.    They will also take legal action. 

Conversation Summary in Hindi: 
सुरेश पर सीएफआई फाइनेंस कंपनी का 75,000 रुपये ईएमआई बकाया है। उन पर कंपनी का 50 हजार रुपये बकाया है। सीएफआई को कल तक 25,000 रुपये चाहिए। सुरेश की मां बीमार हैं और उनके इलाज में पैसे खर्च हो गये. अगर उसने कल तक पैसे नहीं दिए तो सीएफआई उसके घर आएगी और वह पुलिस को बुलाएगा और वे देखेंगे कि कौन उसका इलाज कर सकता है।    वे कानूनी कार्रवाई भी करेंगे.


# Key Action Items

In [18]:
action_tokenize = AutoTokenizer.from_pretrained("debal/bart-large-samsum-action-items")
action_model = AutoModelForSeq2SeqLM.from_pretrained("debal/bart-large-samsum-action-items")

In [20]:
#Actions Item per Sentence Chunk
action_items = []

for conv in en_chunks:
    inputs = action_tokenize([conv], max_length=1024, return_tensors="pt",truncation=True)
    summary_ids = action_model.generate(inputs["input_ids"], num_beams=2, min_length=32, max_length=64)
    action_items.append(action_tokenize.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0])

In [21]:
for i,conv in enumerate(en_chunks):
    print(f'[{hi_chunks[i]}]\n %%{action_items[i]}%%\n')

[ नमस्ते, मैं राजेश बोल रहा हूँ, CFI फाइनेंस कंपनी से। क्या मैं श्री सुरेश कुमार से बात कर सकता हूँ?  हाँ, मैं ही सुरेश बोल रहा हूँ। क्या बात है?  सुरेशजी, मैं आपके लोन के बारे में बात करने के लिए फोन कर रहा हूँ। आपका EMI पिछले तीन महीने से बकाया है।  अरे हाँ, मुझे पता है। लेकिन आप लोग हर रोज फोन करके परेशान कर रहे हैं। मैंने कहा था ना कि जैसे ही पैसे होंगे, मैं भर दूंगा।  देखिए सुरेशजी, हम समझते हैं आपकी परेशानी। लेकिन आप भी समझिए, यह हमारा काम है। आपको पता है कि आपके ऊपर कितना बकाया है?  हाँ-हाँ, मुझे अंदाजा है। लगभग 50,000 रुपये होंगे।  नहीं सुरेशजी, आप गलत हैं। आप पर कुल 75,000 रुपये बकाया है, जिसमें लेट फीस और ब्याज भी शामिल है।  क्या? इतने ज्यादा? यह कैसे हो सकता है? आप लोग लूट रहे हैं क्या?  देखिए सुरेशजी, गुस्सा करने से कुछ नहीं होगा। आपको समय पर पैसे जमा करने चाहिए थे। अब बताइए, कब तक आप यह राशि जमा कर सकते हैं?  अरे भाई, मेरी बात तो सुनो! मेरी माँ की तबीयत खराब है, अस्पताल में भर्ती हैं। सारे पैसे उनके इलाज में खर्च हो गए।]
 %%Action Item:[SEP]Pay outstanding mortgage payment

In [22]:
#Action Items from entire Conversation

inputs = action_tokenize([en_convo], max_length=1024, return_tensors="pt",truncation=True)
summary_ids = action_model.generate(inputs["input_ids"], num_beams=2, min_length=64, max_length=256)
actions = action_tokenize.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

In [23]:
print(actions)

Action Item:[SEP]Deposit Rs 25,000 by tomorrow||Rajesh[SEP]Get mother treated or get loan repayment||Suresh Kumar[SEP]Come to Suresh's house and see who treats whom||UNKNOWN[SEP]Call the police if money doesn't come by tomorrow || Rajesh[SEP]Come and expose SuresH in front of everyone||Raja


# Sentiment Analysis

In [28]:
sentiment_tokenize = AutoTokenizer.from_pretrained("LondonStory/txlm-roberta-hindi-sentiment")
sentiment_analyze = AutoModelForSequenceClassification.from_pretrained("LondonStory/txlm-roberta-hindi-sentiment").to(device)

In [30]:
def sentiment(label):
    if label == 'LABEL_1':
        return 'Negative'
    elif label == 'LABEL_2':
        return 'Positive'
    else:
        return 'Neutral'

In [32]:
sentiments = []

for conv in hi_chunks:
    inputs = sentiment_tokenize(conv, return_tensors="pt").to(device)
    with torch.no_grad():
        output = sentiment_analyze(**inputs).logits
    predicted_class_id = output.argmax().item()
    sentiment_label = sentiment_analyze.config.id2label[predicted_class_id]
    sentiments.append(sentiment(sentiment_label))

In [34]:
#Sentiment of each Chunk of Conversation

for i, conv in enumerate(hi_chunks):
    print(f'{conv}\n%%Sentiment: {sentiments[i]} %%\n')

#Overall Sentiment

item_counts = Counter(sentiments)
print(f'\nOVERALL SENTIMENT OF CONVERSATION: {item_counts.most_common(1)[0][0]}')

 नमस्ते, मैं राजेश बोल रहा हूँ, CFI फाइनेंस कंपनी से। क्या मैं श्री सुरेश कुमार से बात कर सकता हूँ?  हाँ, मैं ही सुरेश बोल रहा हूँ। क्या बात है?  सुरेशजी, मैं आपके लोन के बारे में बात करने के लिए फोन कर रहा हूँ। आपका EMI पिछले तीन महीने से बकाया है।  अरे हाँ, मुझे पता है। लेकिन आप लोग हर रोज फोन करके परेशान कर रहे हैं। मैंने कहा था ना कि जैसे ही पैसे होंगे, मैं भर दूंगा।  देखिए सुरेशजी, हम समझते हैं आपकी परेशानी। लेकिन आप भी समझिए, यह हमारा काम है। आपको पता है कि आपके ऊपर कितना बकाया है?  हाँ-हाँ, मुझे अंदाजा है। लगभग 50,000 रुपये होंगे।  नहीं सुरेशजी, आप गलत हैं। आप पर कुल 75,000 रुपये बकाया है, जिसमें लेट फीस और ब्याज भी शामिल है।  क्या? इतने ज्यादा? यह कैसे हो सकता है? आप लोग लूट रहे हैं क्या?  देखिए सुरेशजी, गुस्सा करने से कुछ नहीं होगा। आपको समय पर पैसे जमा करने चाहिए थे। अब बताइए, कब तक आप यह राशि जमा कर सकते हैं?  अरे भाई, मेरी बात तो सुनो! मेरी माँ की तबीयत खराब है, अस्पताल में भर्ती हैं। सारे पैसे उनके इलाज में खर्च हो गए।
%%Sentiment: Negative %%

  सुरेशजी, मुझे आपकी परिस्थि

# Named Entity Recognition

In [36]:
ner_tokenize = AutoTokenizer.from_pretrained("google/muril-base-cased")
ner_model = AutoModelForTokenClassification.from_pretrained("MichaelHuang/muril_base_cased_hindi_ner")

In [37]:
labels_dict = {
    0: "B-FESTIVAL",
    1: "B-GAME",
    2: "B-LANGUAGE",
    3: "B-LITERATURE",
    4: "B-LOCATION",
    5: "B-MISC",
    6: "B-NUMEX",
    7: "B-ORGANIZATION",
    8: "B-PERSON",
    9: "B-RELIGION",
    10: "B-TIMEX",
    11: "I-FESTIVAL",
    12: "I-GAME",
    13: "I-LANGUAGE",
    14: "I-LITERATURE",
    15: "I-LOCATION",
    16: "I-MISC",
    17: "I-NUMEX",
    18: "I-ORGANIZATION",
    19: "I-PERSON",
    20: "I-RELIGION",
    21: "I-TIMEX",
    22: "O"
}

In [40]:
inputs = ner_tokenize(hi_convo, return_tensors="pt", padding=True, truncation=True, max_length=128)

In [42]:
with torch.no_grad():
  output = ner_model(**inputs).logits

predicted_labels = output.argmax(-1)
labels = predicted_labels.squeeze().tolist()
tokens = ner_tokenize.convert_ids_to_tokens(inputs["input_ids"][0])

In [44]:
predicted_labels = [labels_dict[label] for label in labels]

result = list(zip(tokens, predicted_labels))

for token, label in result:
  if label != "O":
    print(f"{token}: {label}")

राजेश: B-PERSON
CF: B-ORGANIZATION
##I: B-ORGANIZATION
फाइनेंस: I-ORGANIZATION
कंपनी: I-ORGANIZATION
श्री: B-PERSON
सुरेश: B-PERSON
कुमार: I-PERSON
सुरेश: B-PERSON
सुरेश: B-PERSON
तीन: B-TIMEX
महीने: I-TIMEX
सुरेश: B-PERSON


# Non-Compliance Check

In [46]:
compliance = {
    "Using_Threatening_Language": "If you don't pay this debt by tomorrow, we will send someone to your house to collect the money, and it won't be pleasant",
    "Disclosing_Debt_Information_to_Third Parties":"We called your workplace and spoke to your colleague about your debt situation",
    "Calling_Outside_Permissible_Hours": "It's 10 PM, but I need to remind you again to pay your overdue amount",
    "Providing_False_or_Misleading_Information": "If you don't pay this debt immediately, you will be arrested tomorrow",
    "Failing_to_Validate_the_Debt": "You owe $1,000, and there's no need for us to provide proof. Just pay it"
}

In [48]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

In [50]:
for i,speech in enumerate(en_agent_spoke):
    agent_input_id = tokenizer(speech, return_tensors="pt")
    agent_output_id = model(**agent_input_id)
    agent_output = agent_output_id.last_hidden_state[:,0,:]
    issues = []
    for policy in compliance:
        comply_id = tokenizer(compliance[policy], return_tensors="pt")
        comply_output_id = model(**comply_id)
        comply_output = comply_output_id.last_hidden_state[:,0,:]
        if(torch.cosine_similarity(agent_output,comply_output).item()>=0.89):
            issues.append(policy)
    if len(issues):
        print(f'Compliance Issue with "{hi_agent_spoke[i]}"=>{issues}\n')

Compliance Issue with " करो जो करना है। लेकिन याद रखना, अगर कल तक पैसे नहीं आए तो मैं खुद तुम्हारे घर आऊंगा। और तब देखेंगे कौन किसका इलाज करता है।"=>['Using_Threatening_Language']

Compliance Issue with " पहले तो अपनी माँ का इलाज बंद करो। उन पैसों से हमारा कर्ज चुका दो।"=>['Failing_to_Validate_the_Debt']

Compliance Issue with " वो तुम्हारी समस्या है। चाहे दोस्तों से उधार लो, या फिर कुछ बेच दो। लेकिन कल तक पैसे चाहिए।"=>['Failing_to_Validate_the_Debt']

Compliance Issue with " तो फिर ठीक है। कल तक पैसे जमा कर दो। वरना तुम जानते हो क्या होगा।"=>['Failing_to_Validate_the_Debt']

Compliance Issue with " बहुत अच्छा। अब तुम समझदारी की बात कर रहे हो। याद रखना, कल शाम 5 बजे तक पैसे नहीं आए तो मैं सीधा तुम्हारे ऑफिस पहुँच जाऊंगा।"=>['Calling_Outside_Permissible_Hours']

